# Common Libraries

In [1]:
import os
import sys
import csv
import pandas as pd
from glob import glob
from tqdm import tqdm
from pathlib import Path

# Paths

In [2]:
log_file_paths = glob("/home/seojin/temp/*")
duplicated_file_log = "/home/seojin/temp/duplicated.csv"

# Load md5 log results

In [3]:
log_dfs = []
for path in log_file_paths:
    file_name = Path(path).stem
    log_df = pd.read_csv(path)
    log_dfs.append(log_df)
    
total_log_df = pd.concat(log_dfs)
del log_dfs

# Check duplication

In [4]:
is_duplicate = total_log_df["md5"].duplicated()

In [5]:
duplicated_data = total_log_df[is_duplicate]
duplicated_md5s = sorted(list(set(duplicated_data["md5"])))

# Load duplication result log

In [6]:
if not os.path.exists(duplicated_file_log):
    with open(duplicated_file_log, mode = "w", newline = "") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["md5", "n_dup", "sizePerFile", "filePaths"])

exist_indexes = pd.read_csv(duplicated_file_log).index
start_i = 0 if len(exist_indexes) == 0 else exist_indexes[-1] + 1

In [7]:
start_i

718041

# Write duplication result

In [8]:
with open(duplicated_file_log, mode = "a", newline = "") as csvfile:
    writer = csv.writer(csvfile)
    for i in tqdm(range(start_i, len(duplicated_md5s))):
        md5 = duplicated_md5s[i]
        dup_result = total_log_df[total_log_df["md5"] == md5]
        size = os.path.getsize(dup_result.iloc[0]["filePath"])
        file_paths = "__DIV__".join(dup_result["filePath"])
        writer.writerow([md5, len(dup_result), size, file_paths])

0it [00:00, ?it/s]
